# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [3]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [4]:
# TODO: Import the necessary libs
# For example: 
import os
import chromadb
from chromadb.utils import embedding_functions
from chromadb.api.models.Collection import Collection
from lib.agents import Agent
from lib.llm import LLM
from lib.state_machine import StateMachine
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool
from dotenv import load_dotenv
from lib.tooling import Tool
from pydantic import BaseModel
from lib.state_machine import StateMachine, Step, EntryPoint, Termination

In [5]:
# TODO: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [6]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created

chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection("udaplay")

@Tool
def retrieve_game(query: str):
    """
    Tool Docstring:
    Semantic search: Finds most relevant results in the vector DB.

    Args:
        - query: a question about game industry

    Returns:
        A list of dictionaries. Each element contains:
        - Platform: e.g., Game Boy, Playstation 5, Xbox 360
        - Name: Name of the game
        - YearOfRelease: Year the game was released
        - Description: Additional details
    """

    search_results = collection.query(
        query_texts=[query],  
        n_results=5  
    )

    results = []
    documents = search_results["documents"][0]  
    metadatas = search_results["metadatas"][0]  

    for doc, meta in zip(documents, metadatas):
        results.append({
            "Platform": meta.get("Platform", "Unknown"),
            "Name": meta.get("Name", "Unknown"),
            "YearOfRelease": meta.get("YearOfRelease", "Unknown"),
            "Description": doc
        })

    return results



#### Evaluate Retrieval Tool

In [8]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result


@Tool
def evaluate_retrieval(question: str, retrieved_docs: list[dict]):
    """Tool Docstring:
    #    Based on the user's question and on the list of retrieved documents, 
    #    it will analyze the usability of the documents to respond to that question. 
    #    args: 
    #    - question: original question from user
    #    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    #    The result includes:
    #    - useful: whether the documents are useful to answer the question
    #    - description: description about the evaluation result
    """
    """Evaluate if retrieved documents can answer the question."""
    
    context = "\n\n".join([doc.get("Description", "") for doc in retrieved_docs])
    
 
    prompt = f"""
    Evaluate if the following documents are sufficient to answer the question.
    Question: "{question}"
    Documents: {context}
    Provide a detailed explanation and a final judgment: 'useful' or 'not useful'.
    """
    
    
    evaluation = evaluator.invoke(prompt)  
    return evaluation

#### Game Web Search Tool

In [9]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web

@Tool
def game_web_search(question: str):
    """
    Tool Docstring:
    Semantic search: Finds most relevant results from the web.
    
    Args:
        - question: a question about the game industry
    
    Returns:
        A list of search results. Each element contains:
        - Title: Title of the webpage or article
        - URL: Link to the page
        - Snippet: Short description or excerpt
    """
    
    results = tavily_client.search(query=question, top_k=5)  # top 5 results
    
   
    formatted_results = []
    for r in results:
        formatted_results.append({
            "Title": r.get("title", "Unknown"),
            "URL": r.get("url", "Unknown"),
            "Snippet": r.get("snippet", "")
        })
    
    return formatted_results

### Agent

In [10]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed


class GameAgentState(BaseModel):
    query: str
    retrieved_docs: list = []
    evaluation_result: str = ""
    web_results: list = []
    final_answer: str = ""


class Model:
    def __init__(self, name="gpt-5-mini"):
        self.name = name

    def generate(self, prompt: str, instructions: str = "") -> str:
        full_prompt = instructions + "\n" + prompt if instructions else prompt
        # Replace this with actual LLM call if available
        return f"[Model Response]: {full_prompt}"

# Instantiate the model
my_model = Model()


class ChromaGameTool:
    def __init__(self, name="RetrieveGame"):
        self.name = name
    def run(self, query: str):
        return retrieve_game(query)  # your existing Chroma tool function

class EvaluateRetrievalTool:
    def __init__(self, name="EvaluateRetrieval", model=None):
        self.name = name
        self.model = model
    def run(self, question: str, retrieved_docs: list[dict]):
        context = "\n\n".join([doc.get("Description", "") for doc in retrieved_docs])
        prompt = f"""
Evaluate if the following documents are sufficient to answer the question.
Question: "{question}"
Documents: {context}
Provide a detailed explanation and a final judgment: 'useful' or 'not useful'.
"""
        return self.model.generate(prompt)

class GameWebSearchTool:
    def __init__(self, name="GameWebSearch"):
        self.name = name
    def run(self, question: str):
        return game_web_search(question)  # your existing Tavily web search function


retrieve_game_tool = ChromaGameTool()
evaluate_tool = EvaluateRetrievalTool(model=my_model)
web_search_tool = GameWebSearchTool()


entry = EntryPoint()
termination = Termination()


retrieve = Step(
    "retrieve",
    lambda s, r=None: {"retrieved_docs": retrieve_game_tool.run(s["query"])}
)


def evaluate_or_search(state, r=None):
    eval_result = evaluate_tool.run(state["query"], state["retrieved_docs"])
    state["evaluation_result"] = eval_result

    if "useful" in str(eval_result).lower():
        return {"final_answer": "Answer retrieved from documents"}
    else:
        web_results = web_search_tool.run(state["query"])
        state["web_results"] = web_results
        return {"final_answer": web_results}

evaluate = Step("evaluate", evaluate_or_search)

workflow = StateMachine(GameAgentState)
workflow.add_steps([entry, retrieve, evaluate, termination])

workflow.connect(entry, "retrieve")
workflow.connect("retrieve", "evaluate")
workflow.connect("evaluate", termination) 


state = GameAgentState(query="Which Switch game has the highest score?")
run = workflow.run(state.dict())  # convert Pydantic model to dict
final_state = run.get_final_state()


print("Final Answer:", final_state.get("final_answer"))
print("Evaluation Result:", final_state.get("evaluation_result"))
print("Retrieved Docs:", final_state.get("retrieved_docs"))
print("Web Search Results:", final_state.get("web_results"))



/tmp/ipykernel_147/3806797913.py:93: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  run = workflow.run(state.dict())  # convert Pydantic model to dict


[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: evaluate
[StateMachine] Terminating: __termination__
Final Answer: Answer retrieved from documents
Evaluation Result: [Model Response]: 
Evaluate if the following documents are sufficient to answer the question.
Question: "Which Switch game has the highest score?"
Documents: [Nintendo Switch] Mario Kart 8 Deluxe (2017) - An enhanced version of Mario Kart 8, featuring new characters, tracks, and improved gameplay mechanics.

[GameCube] Super Smash Bros. Melee (2001) - A crossover fighting game featuring characters from various Nintendo franchises battling it out in dynamic arenas.

[Nintendo 64] Super Mario 64 (1996) - A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach.

[Super Nintendo Entertainment System (SNES)] Super Mario World (1990) - A classic platformer where Mario embarks on a quest to save Princess Toadsto

In [11]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?


questions = [
    "When Pokémon Gold and Silver was released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for Playstation 5?"
]

for q in questions:
    print(f"\n--- Question: {q} ---")
    
    # Create initial state
    state = GameAgentState(query=q)

  
    run = workflow.run(state.dict())  # convert Pydantic model to dict
    final_state = run.get_final_state()
    
    # Print results
    print("Final Answer:", final_state.get("final_answer"))
    print("Retrieved Docs:", final_state.get("retrieved_docs"))
    print("Evaluation Result:", final_state.get("evaluation_result"))
    print("Web Search Results:", final_state.get("web_results"))


--- Question: When Pokémon Gold and Silver was released? ---
[StateMachine] Starting: __entry__


/tmp/ipykernel_147/1943925116.py:20: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  run = workflow.run(state.dict())  # convert Pydantic model to dict


[StateMachine] Executing step: retrieve
[StateMachine] Executing step: evaluate
[StateMachine] Terminating: __termination__
Final Answer: Answer retrieved from documents
Retrieved Docs: [{'Platform': 'Game Boy Color', 'Name': 'Pokémon Gold and Silver', 'YearOfRelease': 1999, 'Description': '[Game Boy Color] Pokémon Gold and Silver (1999) - Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.'}, {'Platform': 'Game Boy Advance', 'Name': 'Pokémon Ruby and Sapphire', 'YearOfRelease': 2002, 'Description': '[Game Boy Advance] Pokémon Ruby and Sapphire (2002) - Third-generation Pokémon games set in the Hoenn region, featuring new Pokémon and double battles.'}, {'Platform': 'Nintendo 64', 'Name': 'Super Mario 64', 'YearOfRelease': 1996, 'Description': "[Nintendo 64] Super Mario 64 (1996) - A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach."}, {'Platform': 'Super Nintendo Entertainment Syste

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes